# DataAnalysis

## Paquetes

In [42]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

## Funciones

In [43]:
def extract_coordinates(point):
    point = point.strip("()")
    x, y = point.split(", ")
    return int(x), int(y)

## Código

### Lectura de datos

In [44]:
# Read the data from the excel file
data = pd.read_excel('data/output_points_6.xlsx')

# Split columns into two
for column in data.columns:
    data[column + ' (x)'] = data[column].apply(lambda p: extract_coordinates(p)[0])
    data[column + ' (y)'] = data[column].apply(lambda p: extract_coordinates(p)[1])

# Drop the original columns
data = data.drop(columns=data.columns[:6])

# Take Left Point 2 and Right Point 2
data_point_2 = data[['Left Point 2 (x)', 'Left Point 2 (y)', 'Right Point 2 (x)', 'Right Point 2 (y)']]

# Delete rows with 0 values, reindex
data_point_2 = data_point_2[(data_point_2.T != 0).all()]
data_point_2 = data_point_2.reset_index(drop=True)

# Find first row with the value equal to -2
index = data_point_2.index[data_point_2['Left Point 2 (x)'] == -2].tolist()[0]

# Delete rows after the first row with the value equal to -2
data_point_2 = data_point_2.iloc[:index]

# Change -1 values to nan
data_point_2 = data_point_2.replace(-1, np.nan)

# Complete nan by interpolation
data_point_2 = data_point_2.interpolate()

# Delete column Right Point 2 (x)
data_point_2 = data_point_2.drop(columns='Right Point 2 (x)')

# Rename columns: x, y, z
data_point_2.columns = ['x', 'y', 'z']

# Show the data
print(data_point_2.head(20))


        x      y    z
0   412.0  315.0  370
1   377.0  258.0  295
2   342.0  201.0  221
3   300.0  160.0  163
4   273.0  131.0   82
5   257.0  111.0   73
6   265.0  103.0   45
7   269.0  110.0   50
8   267.5  111.0   67
9   266.0  112.0   75
10  268.0  109.0   87
11  263.0  101.0   95
12  263.0   99.0   95
13  260.0   94.0   99


### Gráfico 3D

In [45]:
df = data_point_2.copy()

In [46]:
# Crear los tamaños de los marcadores, haciendo el último punto más grande
marker_sizes = [5] * (len(df) - 1) + [10]  # Último punto con tamaño 10

# Crear el gráfico 3D interactivo
fig = go.Figure()

# Añadir la trayectoria con todos los puntos menos el último
fig.add_trace(go.Scatter3d(
    x=df['x'][:-1],
    y=df['y'][:-1],
    z=df['z'][:-1],
    mode='lines+markers',
    marker=dict(size=2),
    line=dict(width=2),
    name = 'Trayectoria'
))

# Añadir el último punto con un tamaño más grande
fig.add_trace(go.Scatter3d(
    x=[df['x'].iloc[-1]],
    y=[df['y'].iloc[-1]],
    z=[df['z'].iloc[-1]],
    mode='markers',
    marker=dict(size=5, color='red'),  # Color opcional para diferenciar el último punto
    name='Flor'
))

# Configurar el diseño del gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['x']) - 10, max(df['x']) + 10], backgroundcolor="lightblue", gridwidth=0.5),
        yaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['y']) - 10, max(df['y']) + 10], backgroundcolor="lightblue", gridwidth=0.5),
        zaxis=dict(showticklabels=False, tickmode="linear", dtick=50,  range=[min(df['z']) - 10, max(df['z']) + 10], backgroundcolor="lightblue", gridwidth=0.5)
    ),
    title='Trayectoria 3D Interactiva'
)

# Configurar la cámara para que salga cuadrado y alejado de los puntos
camera = dict(
    eye=dict(x=1.5, y=1.5, z=1.5)
)

fig.update_layout(scene_camera=camera) 

# Mostrar el gráfico
fig.show()

### Curvatura

In [47]:
# Convertir a arrays de numpy
x = np.array(df['x'])
y = np.array(df['y'])
z = np.array(df['z'])

# Calcular las derivadas usando diferencias finitas
dx = np.gradient(x)
dy = np.gradient(y)
dz = np.gradient(z)

ddx = np.gradient(dx)
ddy = np.gradient(dy)
ddz = np.gradient(dz)

# Calcular la curvatura
curvatura = np.zeros(len(x))

for i in range(len(x)):
    # Vectores de la primera y segunda derivada
    r_prime = np.array([dx[i], dy[i], dz[i]])
    r_double_prime = np.array([ddx[i], ddy[i], ddz[i]])
    
    # Producto cruzado de r' y r''
    cross_product = np.cross(r_prime, r_double_prime)
    
    # Magnitud de r'
    norm_r_prime = np.linalg.norm(r_prime)
    
    # Curvatura
    curvatura[i] = np.linalg.norm(cross_product) / (norm_r_prime**3)

# Añadir la curvatura al DataFrame
df['curvatura'] = curvatura

# Desviación Estánda de la curvatura
std_curvatura = np.std(curvatura)

# Mostrar la desviación estándar de la curvatura
print('Desviación Estándar de la Curvatura:', std_curvatura)


# Mostrar el DataFrame
print(df.head(20))

Desviación Estándar de la Curvatura: 0.02257447205537303
        x      y    z  curvatura
0   412.0  315.0  370   0.000033
1   377.0  258.0  295   0.000376
2   342.0  201.0  221   0.000982
3   300.0  160.0  163   0.000891
4   273.0  131.0   82   0.001476
5   257.0  111.0   73   0.018159
6   265.0  103.0   45   0.074266
7   269.0  110.0   50   0.043345
8   267.5  111.0   67   0.015797
9   266.0  112.0   75   0.033230
10  268.0  109.0   87   0.026923
11  263.0  101.0   95   0.058601
12  263.0   99.0   95   0.012665
13  260.0   94.0   99   0.014142


### Animación

In [123]:
# Crear el gráfico 3D interactivo
fig = go.Figure(
    data=[
        go.Scatter3d(),
        go.Scatter3d(
            x=[df['x'].iloc[-1]],
            y=[df['y'].iloc[-1]],
            z=[df['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='red'),  # Color opcional para diferenciar el último punto
            name='Flor'
        ),
        go.Scatter3d(
            x=df['x'][:-1],
            y=df['y'][:-1],
            z=df['z'][:-1],
            mode='lines+markers',
            marker=dict(size=0.1, color='blue'),
            line=dict(width=0.1),
            name = 'Trayectoria'
        )
    ]
)


# Agrego los frames
fig.frames = [go.Frame(
    data=[
        go.Scatter3d(
            x=df['x'][:k+1],
            y=df['y'][:k+1],
            z=df['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='black'),
            line=dict(width=4, color='black'),
            name = 'Murciélago'
        )
    ]
) for k in range(len(df))]

# Configurar el diseño del gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['x']) - 15, max(df['x']) + 15], backgroundcolor="lightblue", gridwidth=0.5),
        yaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['y']) - 15, max(df['y']) + 15], backgroundcolor="lightblue", gridwidth=0.5),
        zaxis=dict(showticklabels=False, tickmode="linear", dtick=50,  range=[min(df['z']) - 15, max(df['z']) + 15], backgroundcolor="lightblue", gridwidth=0.5)
    ),
    title='Trayectoria 3D Interactiva',
    updatemenus=[dict(
            type="buttons", 
            y=1.05, x=0.8, xanchor='left', yanchor='bottom',
            buttons=[
                dict(
                    label='Play', method='animate',
                    args=[None, dict(frame=dict(duration=200, redraw=True))]
                )
            ]
    )],
)


# Mostrar el gráfico
fig.show()